In [2]:
import re
import pandas as pd
import numpy as np
from pprint import pprint

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt

# spacy for lemmatization
import spacy
import json
import warnings
import networkx as nx

warnings.filterwarnings("ignore",category=DeprecationWarning)

import nltk
nltk.download('stopwords')
nltk.download('words')

# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

from nltk.corpus import words
eng_words = words.words('en')

from langdetect import detect
# to enforce consistent results, check github langdetect readme
from langdetect import DetectorFactory
DetectorFactory.seed = 0

import os
import requests

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/siyanththan_v/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /home/siyanththan_v/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
df222 = pd.read_csv('mount/finalCSV/rrSet14_22.csv',nrows=100000, lineterminator='\n')
textValue222 = df1.text.values.tolist()
userIdValue222 = df1.userId.values.tolist()
print(type(textValue222[0]))
engText222=[]
text222=[]
for x in range (len(textValue222)):
    if(len(str(textValue222[x])) >=100):
        try:
            if (detect(str(textValue222[x]))=="en"):
                text222=[textValue222[x], int(userIdValue222[x])]
                engText222.append(text222)
        except:
            pass
print(engText222[0:4])

<class 'str'>


In [20]:
df1 = pd.read_csv('mount/finalCSV/sv23_31.csv',nrows=71123, warn_bad_lines=True, error_bad_lines=False, skip_blank_lines=True, low_memory=False)
textValue = df1.text.values.tolist()
userIdValue = df1.userId.values.tolist()
print(type(textValue[0]))
engText=[]
text=[]
for x in range (len(textValue)):
    if(len(str(textValue[x])) >=100):
        try:
            if (detect(str(textValue[x]))=="en"):
                text=[textValue[x], int(userIdValue[x])]
                engText.append(text)
        except:
            pass
print(engText[0:4])

<class 'str'>
[["“@amychin: Just got caught up on #USAin90days! I can't believe how far ya'll have gone! It's wild  we've just got to Oregon!” // Thanks Amy!", 216566284], ['RT @MMFlint: 44 million functional illiterates in the USA! It\'s easy to manipulate ppl w/ fear if u can keep them stupid thru "schools"  ...', 125724807], ["10 Reasons Why It's Great to be a Dog\n\n1.If it itches  you can reach it. And no matter where it itches  no one... http://t.co/UFobDQMF", 505593695], ["@JDylan_ I know I didn't. I did nothing but laundry today and see my family. I don't get many days off so its very hard for me to make time", 66291383]]


In [21]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

data = engText

# sent[:][0]),sent[:][1]] for sent in data]

# Remove links
data = [[re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '', sent[:][0]),sent[:][1]] for sent in data]

# Remove new line characters
data = [[re.sub('\s+', ' ', sent[:][0]), sent[:][1]] for sent in data]

# Remove distracting single quotes
data = [[re.sub("\'", "", sent[:][0]),sent[:][1]] for sent in data]

# Remove hashtag
data = [[re.sub("#", "", sent[:][0]),sent[:][1]] for sent in data]

print(data[0:4])

[['“@amychin: Just got caught up on USAin90days! I cant believe how far yall have gone! Its wild weve just got to Oregon!” // Thanks Amy!', 216566284], ['RT @MMFlint: 44 million functional illiterates in the USA! Its easy to manipulate ppl w/ fear if u can keep them stupid thru "schools" ...', 125724807], ['10 Reasons Why Its Great to be a Dog 1.If it itches you can reach it. And no matter where it itches no one... ', 505593695], ['@JDylan_ I know I didnt. I did nothing but laundry today and see my family. I dont get many days off so its very hard for me to make time', 66291383]]


In [22]:
data_words = []
data_userId = []

for sentence in data:
    data_words.append(gensim.utils.simple_preprocess(str(sentence[:][0]), deacc=True))  # deacc=True removes punctuations
    data_userId.append(sentence[:][1])
    
print(data_words[0:4])
print(data_userId[0:4])

[['amychin', 'just', 'got', 'caught', 'up', 'on', 'usain', 'days', 'cant', 'believe', 'how', 'far', 'yall', 'have', 'gone', 'its', 'wild', 'weve', 'just', 'got', 'to', 'oregon', 'thanks', 'amy'], ['rt', 'mmflint', 'million', 'functional', 'illiterates', 'in', 'the', 'usa', 'its', 'easy', 'to', 'manipulate', 'ppl', 'fear', 'if', 'can', 'keep', 'them', 'stupid', 'thru', 'schools'], ['reasons', 'why', 'its', 'great', 'to', 'be', 'dog', 'if', 'it', 'itches', 'you', 'can', 'reach', 'it', 'and', 'no', 'matter', 'where', 'it', 'itches', 'no', 'one'], ['jdylan_', 'know', 'didnt', 'did', 'nothing', 'but', 'laundry', 'today', 'and', 'see', 'my', 'family', 'dont', 'get', 'many', 'days', 'off', 'so', 'its', 'very', 'hard', 'for', 'me', 'to', 'make', 'time']]
[216566284, 125724807, 505593695, 66291383]


In [23]:
data_words_no_stp = remove_stopwords(data_words)
print(data_words_no_stp[0:4])

[['amychin', 'got', 'caught', 'usain', 'days', 'cant', 'believe', 'far', 'yall', 'gone', 'wild', 'weve', 'got', 'oregon', 'thanks', 'amy'], ['rt', 'mmflint', 'million', 'functional', 'illiterates', 'usa', 'easy', 'manipulate', 'ppl', 'fear', 'keep', 'stupid', 'thru', 'schools'], ['reasons', 'great', 'dog', 'itches', 'reach', 'matter', 'itches', 'one'], ['jdylan_', 'know', 'didnt', 'nothing', 'laundry', 'today', 'see', 'family', 'dont', 'get', 'many', 'days', 'hard', 'make', 'time']]


In [24]:
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

nlp = spacy.load('en', disable=['parser', 'ner'])

data_lemmatized = lemmatization(data_words_no_stp, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized)

[['get', 'catch', 'usain', 'day', 'can', 'not', 'believe', 'far', 'go', 'wild', 'have', 'get', 'oregon', 'thank', 'amy'], ['mmflint', 'functional', 'illiterate', 'usa', 'easy', 'manipulate', 'ppl', 'fear', 'keep', 'stupid', 'school'], ['reason', 'great', 'dog', 'itch', 'reach', 'matter', 'itch'], ['jdylan', '_', 'know', 'do', 'not', 'nothing', 'laundry', 'today', 'see', 'family', 'do', 'not', 'get', 'many', 'day', 'hard', 'make', 'time'], ['jdbeverywhere', 'critican', 'odian', 'lo', 'insultan', 'pero', 'nadie', 'logra', 'superarlo', 'kidrauhl', 'winner'], ['bukquote', 'want', 'overthrow', 'government', 'bring', 'somebody', 'wife', 'charle', 'bukowski', 'poem', 'title'], ['ramadan', 'buzz', 'e', 'facebook', 'twitter', 'flickr', 'youtube', 'amp', 'buzz', 'e', 'com', 'ramadan'], ['massive', 'stimulus', 'plan', 'exact', 'heavy', 'cost', 'economist', 'headline', 'fb'], ['can', 'not', 'sleep', 'be', 'hella', 'awake', 'can', 'not', 'stop', 'laugh', 'reason', 'mom', 'say', 'need', 'medication'

In [32]:
def remove_non_english_words(tweets):
    x=[]
    for tweet in tweets:
        for word in simple_preprocess(str(tweet)):
            if word in eng_words:
                x.append([word])
    return x
    
dataFinal = remove_non_english_words(data_lemmatized)

In [ ]:
def remove_non_english_words(tweets):
     return [[word for word in simple_preprocess(str(tweet)) if word in eng_words] for tweet in tweets]